In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from utils.data_loader import Dataset
from utils.helpers import * 

from sklearn.svm import SVR 

## TODO:
1. create a benchmark for comparing the results 
2. measure the importance of features based on 

In [5]:
training_filenames = ['./data/cc_training_v2.csv', './data/training_fluid_intelligenceV1.csv', './data/btsv01.txt', \
                     'data/training_data_means.csv', 'data/training_data_entropy.csv',  'data/training_data_stdevs.csv',]
validation_filenames=['./data/cc_validation_v2.csv', './data/validation_fluid_intelligenceV1.csv', './data/btsv01.txt', \
                     'data/validation_data_means.csv', 'data/validation_data_entropy.csv', 'data/validation_data_stdevs.csv']

cols_to_drop = ['btsv01_id', 'interview_date', 'collection_id', 'dataset_id', 'collection_title', \
                'src_subject_id', 'gender']

label_col = 'residual_fluid_intelligence_score'

# training = Dataset(training_filenames, cols_to_drop, label_col)
# validation = Dataset(validation_filenames, cols_to_drop, label_col)

In [ ]:
custom_columns = []
#here we generate custom columns - most of them rely on calculating ratios, so we will provide a function based on that 

#here we append to our existing train data 


In [ ]:
#here think about how we will benchmark the data
models = {
    'majority': [np.mean(train_data) for x in train_data], 
    'random': np.random.rand(len(train_data))*15, #15 is a sd of our distribution
    'svr': SVR(),
    'randfor': RandomForestRegressor(), 
    'xgboost': XGBoostRegressor()
}
#list - each element is an object of a model that we want to try


def restrict_dataset(df, df_dict, *filenames):
    #here we restrict our dataset to only columns that are found in the dataframes from the following filenames
    valid_cols = []
    for file in filenames:
        valid_cols.append(df_dict[file]['columns'])
    valid_cols = set(df.columns).intersection(set(valid_cols))
    return df[[valid_cols]]


#here we will plot the results for each combination of features 

